## Entrenamiento de una Red Neuronal para el Reconocimiento de Imágenes

Como muchos saben, **Deep Learning** está tomando mucha importancia hoy en día, siendo sin lugar a dudas uno de los campos del aprendizaje de máquina más popular. Este puede ser entendido como un grupo de algoritmos que fueron desarrollados para entrenar redes neuronales con muchos niveles de una manera muy eficiente. En este trabajo se desarrollará el concepto básico de las redes neuronales y se cubrirá los siguientes puntos:

* Concepto de una red neuronal multi-capa.
* Entrenar redes neuronales para la clasificación de imágenes.
* Implementacón del algoritmo Backpropagation.
* Debugging de las implementaciones de una red neuronal.

En la última década, muchas más mejoras han sido descubiertas dentro de los algoritmos de Deep Learning, los cuales pueden para crear detectores para los datos usados para entrenar redes neuronales profundas, que están compuestas por muchas capas.

### Red Neuronal de una sola capa (Algoritmo ADALINE)

![Adaline](imagenes/1Capa.png "Adaline")

In [1]:
import numpy as np
#los valores de retorno se guardan dentro del objeto perceptron
class Perceptron(object):
    def __init__(self, eta=0.01, n_iter=10):
        self.eta = eta
        self.n_iter = n_iter

    def entrenar_perceptron(self, X, y):
        self.w_ = np.zeros(1 + X.shape[1])
        self.errors_ = []

        for _ in range(self.n_iter):
            errors = 0
            for xi, target in zip(X, y):
                update = self.eta * (target - self.prueba_perceptron(xi))
                self.w_[1:] += update * xi
                self.w_[0] += update
                errors += int(update != 0.0)
            if errors==0.0:
                break
            self.errors_.append(errors)
        return self

    def net_input(self, X):
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def prueba_perceptron(self, X):
        return np.where(self.net_input(X) >= 0.0, 1, -1)

#### Introducción a una red neuronal multi-capa (MLP)
La siguiente figura explica el concepto de una MLP que consist en 3 capas: una capa de entrada, una capa escondida, y una capa de salida. Las unidades en las capas escondidas están totalmente conectadas con la capa de entrada, y la capa de salida está totalmente conectada con la capa escondida. Si una red tiene más de 1 capa escondida, la llamaremos red neuronal profunda.

![MLP](imagenes/3Capas.png "MLP")

#### Forward propagation
Resumiendo el aprendizaje de un MLP:
* Comienza con la capa de entrada, donde se propaga los patrones de los datos de entrenamiento a trav[es de la red para generar un resultado.
* Basado en el resultado de la red, calculamos el error que se quiere minimizar usando una función de costo que se describirá luego.
* Se propaga el error, se encuentra la diferencia con respecto a cada peso en la red, y se actualiza el modelo.

#### Red Neuronal Convolucional (CNNs)

Son las responsables de la mayoría de los avances en reconocimiento de imágenes hechos en los últimos años.

* Implementación de un CNN simple.
* Implementación de un CNN avanzado.
* Re-entrenamiento de un modelo existente CNN.

#### Introducción:

En matemáticas, una convolución es una función, la cual es aplicada sobre el resultado de otra función. En nuestro caso, nosotros consideraremos aplicar una multiplicación de matrices (filtro) sobre una imagen.

![Convolution](imagenes/Convolution.png "Convolution")

Las redes neuronales convolucionales también tienen otras operaciones que cumplen otras necesidades, como introducir no linealidades (ReLU), o agregar parámetros (maxpool), y otras operaciones similares. La imagen anterior es un ejemplo de aplicar la operación de convolución en una matrix 5x5 con un filtro convolucional de dimensión 2x2. El tamaño del paso es 1 y solo consideramos ubicaciones válidas. Las variables entrenables en esta operación serían los pesos de filtro de 2x2.
Después de una convolución, es común hacer un seguimiento con un agregado, llamado maxpool. El siguiente diagrama proporciona un ejemplo de cómo opera maxpool:

![Maxpool](imagenes/Maxpool.png "Maxpool")

Es común tomar una red pre-entrenada y re-entrenarla con un nuevo conjunto de datos junto a una capa completamente conectada al final. Este método es muy beneficioso.

#### Re-entrenar modelos CNNs

Entrenar un nuevo reconocedor de imágenes desde cero, requiere mucho tiempo y poder de computo. Si nosotros tomamos una red entrenada y la re-entrenamos con nuestras imágenes, esto puede ahorrarnos tiempo de cómputo. La idea es reusar los pesos y la estructura de un modelo de capas convolucionales y re-entrenarlo con unas capas totalmente conectadas en la cima de la red.

#### Aplicación

Una vez que se tenga la CNN entrenada, se podrá usar la red para el procesamiento de imágenes. Primero tendrá que aprender el estilo de una imagen y aplicarla a otra mientras que la segunda mantiene su estructura (o contenido). Esto será posible si se encuentran los nodos intermedios de la CNN que relacionen al estilo con el contenido de la imagen.

Primero se descargará un modelo pre-entrenado de la red: [http://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat](http://www.vlfeat.org/matconvnet/models/beta16/imagenet-vgg-verydeep-19.mat)

Luego se cargarán las librerías necesarias:

In [1]:
import os
import scipy.misc
import scipy.io
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()

/home/rogger/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Se inicia una sesión para el grafo.

In [21]:
sess = tf.Session()

In [22]:
# Imagenes para cargar
original_image_file = 'imagenes/book_cover.jpg'
style_image_file = 'imagenes/Crash.jpg'

# Modelo VGG
vgg_path = 'imagenet-vgg-verydeep-19.mat'

# Argumentos por defecto
original_image_weight = 5.0
style_image_weight = 500.0
regularization_weight = 100
learning_rate = 0.001
generations = 500
output_generations = 25
beta1 = 0.9   # For the Adam optimizer
beta2 = 0.999 # For the Adam optimizer

In [23]:
# Leer las imagenes
original_image = scipy.misc.imread(original_image_file)
style_image = scipy.misc.imread(style_image_file)

# Obtener el tamaño de la imagen que obtendrá el estilo y cambiar el tamaño de la otra para que ambas tengan el mismo
target_shape = original_image.shape
style_image = scipy.misc.imresize(style_image, target_shape[1] / style_image.shape[1])

In [24]:
# VGG-19 Capas
# Del paper
vgg_layers = ['conv1_1', 'relu1_1',
              'conv1_2', 'relu1_2', 'pool1',
              'conv2_1', 'relu2_1',
              'conv2_2', 'relu2_2', 'pool2',
              'conv3_1', 'relu3_1',
              'conv3_2', 'relu3_2',
              'conv3_3', 'relu3_3',
              'conv3_4', 'relu3_4', 'pool3',
              'conv4_1', 'relu4_1',
              'conv4_2', 'relu4_2',
              'conv4_3', 'relu4_3',
              'conv4_4', 'relu4_4', 'pool4',
              'conv5_1', 'relu5_1',
              'conv5_2', 'relu5_2',
              'conv5_3', 'relu5_3',
              'conv5_4', 'relu5_4']

In [25]:
# Se extrae la información del archivo mat
def extract_net_info(path_to_params):
    vgg_data = scipy.io.loadmat(path_to_params)
    normalization_matrix = vgg_data['normalization'][0][0][0]
    mat_mean = np.mean(normalization_matrix, axis=(0,1))
    network_weights = vgg_data['layers'][0]
    return(mat_mean, network_weights)

In [26]:
# Se recrea la red
def vgg_network(network_weights, init_image):
    network = {}
    image = init_image

    for i, layer in enumerate(vgg_layers):
        if layer[0] == 'c':
            weights, bias = network_weights[i][0][0][0][0]
            weights = np.transpose(weights, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            conv_layer = tf.nn.conv2d(image, tf.constant(weights), (1, 1, 1, 1), 'SAME')
            image = tf.nn.bias_add(conv_layer, bias)
        elif layer[0] == 'r':
            image = tf.nn.relu(image)
        else:
            image = tf.nn.max_pool(image, (1, 2, 2, 1), (1, 2, 2, 1), 'SAME')
        network[layer] = image
    return(network)

In [27]:
original_layer = 'relu4_2'
style_layers = ['relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1']

In [28]:
normalization_mean, network_weights = extract_net_info(vgg_path)

shape = (1,) + original_image.shape
style_shape = (1,) + style_image.shape
original_features = {}
style_features = {}

In [29]:
# Se recrea la red
image = tf.placeholder('float', shape=shape)
vgg_net = vgg_network(network_weights, image)

In [30]:
# Se normaliza la matriz de la imagen y se corre sobre la red
original_minus_mean = original_image - normalization_mean
original_norm = np.array([original_minus_mean])
original_features[original_layer] = sess.run(vgg_net[original_layer], feed_dict={image: original_norm})

In [31]:
# Se repite el procedimiento con cada capa de estilo que se escogió antes
image = tf.placeholder('float', shape=style_shape)
vgg_net = vgg_network(network_weights, image)
style_minus_mean = style_image - normalization_mean
style_norm = np.array([style_minus_mean])

for layer in style_layers:
    layer_output = sess.run(vgg_net[layer], feed_dict={image: style_norm})
    layer_output = np.reshape(layer_output, (-1, layer_output.shape[3]))
    style_gram_matrix = np.matmul(layer_output.T, layer_output) / layer_output.size
    style_features[layer] = style_gram_matrix

In [32]:
# Para crear la combinación, se agrega ruido a través de la red
initial = tf.random_normal(shape) * 0.256
image = tf.Variable(initial)
vgg_net = vgg_network(network_weights, image)

In [33]:
# Primera perdida
original_loss = original_image_weight * (2 * tf.nn.l2_loss(vgg_net[original_layer] - original_features[original_layer]) /
                original_features[original_layer].size)

# Perdida en el estilo
style_loss = 0
style_losses = []
for style_layer in style_layers:
    layer = vgg_net[style_layer]
    feats, height, width, channels = [x.value for x in layer.get_shape()]
    size = height * width * channels
    features = tf.reshape(layer, (-1, channels))
    style_gram_matrix = tf.matmul(tf.transpose(features), features) / size
    style_expected = style_features[style_layer]
    #style_temp_loss = sess.run(2 * tf.nn.l2_loss(style_gram_matrix - style_expected) / style_expected.size)
    #print('Layer: {}, Loss: {}'.format(style_layer, style_temp_loss))
    style_losses.append(2 * tf.nn.l2_loss(style_gram_matrix - style_expected) / style_expected.size)
style_loss += style_image_weight * tf.reduce_sum(style_losses)

# Se calcula la variación total       
total_var_x = sess.run(tf.reduce_prod(image[:,1:,:,:].get_shape()))
total_var_y = sess.run(tf.reduce_prod(image[:,:,1:,:].get_shape()))
first_term = regularization_weight * 2
second_term_numerator = tf.nn.l2_loss(image[:,1:,:,:] - image[:,:shape[1]-1,:,:])
second_term = second_term_numerator / total_var_y
third_term = (tf.nn.l2_loss(image[:,:,1:,:] - image[:,:,:shape[2]-1,:]) / total_var_x)
total_variation_loss = first_term * (second_term + third_term)

# Se combinan las pérdidas
loss = original_loss + style_loss + total_variation_loss

In [34]:
style_layer = 'relu2_1'
layer = vgg_net[style_layer]
feats, height, width, channels = [x.value for x in layer.get_shape()]
size = height * width * channels
features = tf.reshape(layer, (-1, channels))
style_gram_matrix = tf.matmul(tf.transpose(features), features) / size
style_expected = style_features[style_layer]
style_losses.append(2 * tf.nn.l2_loss(style_gram_matrix - style_expected) / style_expected.size)

In [35]:
# Declaracion del algoritmo de Optimización
optimizer = tf.train.AdamOptimizer(learning_rate, beta1, beta2)
train_step = optimizer.minimize(loss)

# Inicialización de varibles y comenzar el entrenamiento
sess.run(tf.global_variables_initializer())

In [36]:
for style_layer in style_layers:
    print('-------Layer: {} -------'.format(style_layer))
    layer = vgg_net[style_layer]
    print(sess.run(layer))
    feats, height, width, channels = [x.value for x in layer.get_shape()]
    size = height * width * channels
    print(size)
    print('')

-------Layer: relu1_1 -------
[[[[ 0.99769211  0.124571    0.23778097 ...,  1.10440874  0.95318878
     1.06157756]
   [ 1.35292888  0.09873338  0.13368434 ...,  1.01553416  0.70510364
     0.82214618]
   [ 0.88811225  0.02323176  0.06103561 ...,  1.06558287  0.61542916
     0.61561382]
   ..., 
   [ 0.57664013  0.08556816  0.         ...,  1.16084933  0.59294796
     0.33717477]
   [ 0.60938329  0.13984084  0.11584599 ...,  1.15865004  0.77761126
     0.66960132]
   [ 0.88127607  0.10543361  0.02637555 ...,  1.05082774  0.40969166
     0.38527912]]

  [[ 1.37836874  0.          0.05851355 ...,  1.05180013  0.91608578
     0.97667742]
   [ 1.25047779  0.          0.         ...,  1.01488149  0.74933147
     0.68917441]
   [ 0.43000606  0.          0.         ...,  1.00617182  0.65972489
     0.46168712]
   ..., 
   [ 0.34920537  0.10079706  0.         ...,  1.09308922  0.157204    0.        ]
   [ 0.48915446  0.1032795   0.08191188 ...,  1.16370654  0.67862147
     0.54354072]
   [ 0.8

[[[[  6.56107855  10.50358582   0.         ...,   0.          45.85224915
      0.14670382]
   [  8.89608765  13.018363     0.         ...,   0.          16.65531921
      6.33728981]
   [  6.95187998   7.90370703   0.         ...,   0.           4.3931694
     11.6180315 ]
   ..., 
   [  3.5654788   16.57735825   0.         ...,   0.          29.97684097
     12.9567852 ]
   [  9.40552235  27.66011429   0.         ...,   0.           0.
     12.60405636]
   [ 10.26278591  28.83856964   2.77758193 ...,   0.           0.
     22.41108513]]

  [[  0.05949891  15.61852741   0.         ...,   0.          52.58604813
      0.        ]
   [  0.          22.69583702   3.55708003 ...,   0.          11.73867989
      0.        ]
   [ 10.39736557  19.79232025   0.         ...,   0.           8.2525816
      0.        ]
   ..., 
   [  6.18919468  30.29744148   0.         ...,   0.          28.1273098
      0.        ]
   [  0.          48.10041046   0.         ...,   0.           0.           0. 

In [37]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_step = optimizer.minimize(loss)

sess.run(tf.global_variables_initializer())
for i in range(generations):
    print("generacion ", i)
    sess.run(train_step)

    # Imprimir las actulizaciones y guardar imágenes temporales
    if (i+1) % output_generations == 0:
        print('Generation {} out of {}, loss: {}'.format(i + 1, generations,sess.run(loss)))
        image_eval = sess.run(image)
        best_image_add_mean = image_eval.reshape(shape[1:]) + normalization_mean
        output_file = 'temp_output_{}.jpg'.format(i)
        scipy.misc.imsave(output_file, best_image_add_mean)

generacion  0
generacion  1
Generation 2 out of 10, loss: 310105984.0
generacion  2
generacion  3
Generation 4 out of 10, loss: 310080288.0
generacion  4
generacion  5
Generation 6 out of 10, loss: 310050240.0
generacion  6
generacion  7
Generation 8 out of 10, loss: 310015040.0
generacion  8
generacion  9
Generation 10 out of 10, loss: 309973472.0


In [38]:
image_eval = sess.run(image)
best_image_add_mean = image_eval.reshape(shape[1:]) + normalization_mean
output_file = 'final_output.jpg'
scipy.misc.imsave(output_file, best_image_add_mean)